In [29]:
import requests
import json 
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [30]:
def querystring(start=0, elems=100) -> dict:
    return {"q":"alfa giulia","c":"2","t":"s","qso":"false","shp":"false","urg":"false","sort":"datedesc","lim":f"{elems}","start":f"{start}"}

In [31]:
url = "https://hades.subito.it/v1/search/items"

payload = ""
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7",
    "origin": "https://www.subito.it",
    "priority": "u=1, i",
    "referer": "https://www.subito.it/",
    "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    "sec-ch-ua-mobile": "?1",
    "sec-ch-ua-platform": '"Android"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Mobile Safari/537.36",
    "x-subito-channel": "web",
    "x-subito-environment-id": "c79e6360-ea6b-431f-9b65-3be9d52c7b84"
}

In [32]:
try:
    i = 0
    elems = 100
    data_rows = []
    
    more_results = True

    while more_results:
        response = requests.request("GET", url, data=payload, headers=headers, params=querystring(i, elems))

        json_response = response.json()  # Parse response text as JSON
        # Print the keys of the JSON response
        ads = json_response['ads']

        if len(ads) < elems:
            more_results = False

        for ad in ads:
            # Initialize a row with the necessary fields
            row = [
                ad.get('subject', ''),
                ad.get('body', ''),
                ad['advertiser'].get('name', ''),
                ad['advertiser'].get('company', ''),
                ad['geo']['city'].get('value', ''),
                ad['geo']['city'].get('short_name', ''),
            ]
            
            features = ad.get('features', [])
            
            # Extract specific features
            feature_values = {
                'Doors': None,
                'Fuel': None,
                'Vehicle_status': None,
                'Vat_deductible': None,
                'Gearbox': None,
                'Price': None,
                'Car_type': None,
                'Seats': None,
                'Mileage': None,
                'Mileage_scalar': None,
                'Year': None,
                'Month': None,
            }
            
            for feat in features:
                if feat['uri'] in ['/doors', '/fuel', '/vehicle_status', '/vat_deductible', '/gearbox', 
                                '/price', '/car_type', '/seats', '/mileage', '/mileage_scalar', 
                                '/year', '/month']:
                    label = feat['uri'].replace('/', '').capitalize()
                    value = feat['values'][0]['value']

                    if feat['uri'] == '/mileage_scalar':
                        value = float(value.replace(' Km', '').replace('.', ''))
                    if feat['uri'] == '/price':
                        value = float(value.replace(' €', '').replace('.', ''))
                    
                    feature_values[label] = value
                
                if feat['uri'] == '/car':
                    for value in feat['values']:
                        feature_values[value["label"]] = value['value']
            
            # Extend the row with feature values
            row.extend(feature_values.values())
            if len(row) < 21:
                # aggiungo None alla versione, se non c'è 
                row.append(None)

            # Append the row to the list
            data_rows.append(row)
        i += elems
        
    # Convert list of rows to NumPy array
    np_array = np.array(data_rows, dtype=object)
    #print(np_array)

    columns = [
    'Subject', 'Body', 'Advertiser Name', 'Company', 
    'City', 'Province', 
    'Porte', 'Carburante', 'Tipo di veicolo', 'Iva deducibile', 
    'Cambio', 'Prezzo', 'Tipo di macchina', 'Posti', 
    'Km range', 'Km precisi', 'Anno di immatricolazione', 
    'Mese di immatricolazione', 'Marca', 'Modello', 'Versione'
    ]

    df = pd.DataFrame(np_array, columns=columns)
        
except json.JSONDecodeError:
    print("Failed to parse JSON response")

In [33]:
df

,Subject,Body,Advertiser Name,Company,City,Province,Porte,Carburante,Tipo di veicolo,Iva deducibile,Cambio,Prezzo,Tipo di macchina,Posti,Km range,Km precisi,Anno di immatricolazione,Mese di immatricolazione,Marca,Modello,Versione
0,Alfa Giulia 2.2 180cv AT8,Vendo Alfa Giulia 2.2 180cv super 2016 con 199...,Danilo,False,Agrigento,AG,4/5,Diesel,Usato,Sì,Automatico,16900.0,Berlina,5,190.000 - 199.999,199000.0,2016,Agosto,ALFA ROMEO,Giulia (2016),Giulia 2.2 Turbodiesel 180 CV AT8 Super
1,Alfa romeo Giulia nuova super,"Motore nuovo ,interni e carrozzeria ok,numeros...",Dom,False,Macerata,MC,None,None,Usato,No,None,9200.0,None,None,0 - 4.999,1500.0,1977,Agosto,ALFA ROMEO,Giulia (2016),None
2,ALFA ROMEO Giulia (2016) - 2017,Vendo alfa Romeo Giulia 180 cv con cambio manu...,Marco,False,Sassari,SS,4/5,Diesel,Usato,No,None,20000.0,None,5,180.000 - 189.999,180000.0,2017,Febbraio,ALFA ROMEO,Giulia (2016),Giulia 2.2 Turbodiesel 180 CV Super
3,Alfa Romeo Giulia 2.2 Turbodiesel 150 CV AT8 -...,cilindrata: 2.2 TD 150 cv\ncarburante: diesel\...,LO TRUGLIO AUTO,True,Trapani,TP,4/5,Diesel,Usato,None,Automatico,18900.0,Berlina,5,120.000 - 129.999,127000.0,2017,Dicembre,ALFA ROMEO,Giulia (2016),Giulia 2.2 Turbodiesel 150 CV AT8
4,Alfa Romeo Giulia,Alfa Romeo Giulia versione 2.2 Turbodiesel 150...,Andrea,False,Roma,RM,4/5,Diesel,Usato,None,Automatico,10500.0,Berlina,5,450.000 - 499.999,475000.0,2016,Novembre,ALFA ROMEO,Giulia (2016),Giulia 2.2 Turbodiesel 150 CV Business Launch ...
5,Alfa Romeo Giulia 2.2jtdm SUPER 2018 FINANZIABILE,"Evucarsrl.it vi presenta:\n\nVETTURA: ""Alfa Ro...",EVUCAR SRL,True,Messina,ME,4/5,Diesel,Usato,None,Manuale,19500.0,Berlina,5,110.000 - 119.999,110000.0,2018,Luglio,ALFA ROMEO,Giulia (2016),Giulia 2.2 Turbodiesel 136 CV Business
6,Alfa Romeo Giulia 2.2 Turbodiesel 150 CV Busin...,Veicolo in ordine e tagliandato\nPrezzo passag...,ANCONA S.R.L.,True,Ancona,AN,4/5,Diesel,Usato,None,Manuale,17990.0,Berlina,5,120.000 - 129.999,125000.0,2017,Gennaio,ALFA ROMEO,Giulia (2016),Giulia 2.2 Turbodiesel 150 CV Business Launch ...
7,ALFA ROMEO Altro modello - 1969,Alfa romeo Giulia 1300 TI Anno 1969 Km 72722 C...,nadia,False,Roma,RM,4/5,Benzina,Usato,No,Manuale,11900.0,Berlina,5,70.000 - 74.999,72722.0,1969,Febbraio,ALFA ROMEO,Altro modello,None
8,Alfa Romeo Giulia 2.2 Turbodiesel 150 CV,Auto in buono stato\nMotore con Km 83000 in qu...,Del Magno Auto,True,Rimini,RN,4/5,Diesel,Usato,None,Manuale,13900.0,Berlina,5,200.000 - 249.999,223000.0,2016,Ottobre,ALFA ROMEO,Giulia (2016),Giulia 2.2 Turbodiesel 150 CV
9,Alfa Romeo Giulia 2.2 Turbodiesel 150 CV AT8 B...,Dielle autoservice propone alfa romeo giulia 1...,DIELLE AUTOSERVICE S.N.C. DI DURAS GIOVANNI BATTI,True,Nuoro,NU,4/5,Diesel,Usato,None,Automatico,27950.0,Berlina,5,15.000 - 19.999,19800.0,2017,Settembre,ALFA ROMEO,Giulia (2016),Giulia 2.2 Turbodiesel 150 CV AT8 Business
